## Transfer Learning

**Transfer Learning (ou Aprendizagem por Transferência)** é uma técnica em Machine Learning e Deep Learning onde um modelo treinado em uma tarefa é reutilizado como o ponto de partida para um modelo em uma segunda tarefa relacionada. Em vez de treinar um modelo do zero, o que pode ser muito demorado e requerer grandes quantidades de dados, Transfer Learning aproveita o conhecimento já adquirido em um modelo pré-treinado para acelerar e melhorar o desempenho em uma nova tarefa.

### Funcionamento do Transfer Learning

1. **Modelo Pré-treinado**: Um modelo é treinado em um grande conjunto de dados e em uma tarefa específica (por exemplo, a classificação de imagens no ImageNet).
2. **Transferência do Conhecimento**: As camadas iniciais desse modelo capturam características genéricas das imagens (como bordas, texturas, formas), enquanto as camadas finais são mais específicas para a tarefa original. No Transfer Learning, essas camadas iniciais são reutilizadas.
3. **Adaptação**: O modelo é então ajustado para a nova tarefa, que pode envolver:
   - **Congelar** algumas das camadas iniciais e treinar apenas as últimas camadas com o novo conjunto de dados.
   - **Treinamento Fino (Fine-tuning)**, onde todo o modelo é treinado novamente com uma taxa de aprendizado mais baixa.

### Aplicação Prática

Neste projeto, estamos utilizando um modelo pré-treinado em um dataset bem conhecido (como o ImageNet) e adaptando esse modelo para classificar imagens de gatos e cachorros. A abordagem geral inclui os seguintes passos:

1. **Carregar o modelo pré-treinado**: Utilizando bibliotecas como Keras, TensorFlow ou PyTorch para carregar um modelo que já foi treinado em um grande conjunto de dados.
2. **Modificar a arquitetura**: Adaptar a última camada do modelo para ter o mesmo número de classes que o nosso problema (neste caso, duas classes: gatos e cachorros).
3. **Congelar camadas**: Congelar algumas das primeiras camadas do modelo para preservar as características genéricas aprendidas.
4. **Treinar o modelo**: Usar nosso conjunto de dados específico (gatos e cachorros) para ajustar as camadas finais do modelo e refinar o desempenho.

### Exemplo no COLAB

O exemplo no link [Transfer Learning com o Dataset MNIST](https://colab.research.google.com/github/kylemath/ml4a-guides/blob/master/notebooks/transfer-learning.ipynb) demonstra como aplicar Transfer Learning usando o dataset MNIST. Para este projeto, seguimos uma abordagem similar, mas adaptando para o dataset de gatos e cachorros, conforme descrito no link [Cats vs Dogs Dataset](https://www.tensorflow.org/datasets/catalog/cats_vs_dogs).

### Benefícios do Transfer Learning

- **Redução do Tempo de Treinamento**: O modelo já tem um bom entendimento das características básicas das imagens, então o tempo necessário para treinar é reduzido.
- **Menos Dados Necessários**: Como o modelo já foi treinado em uma grande quantidade de dados, a quantidade de dados necessária para a nova tarefa é menor.
- **Melhor Desempenho**: Utilizando um modelo pré-treinado que já foi ajustado para capturar características relevantes, o desempenho na nova tarefa pode ser melhor do que treinar um modelo do zero.

### Exemplos Comuns

Transfer Learning é amplamente utilizado em várias áreas, como:
- **Visão Computacional**: Classificação de imagens, detecção de objetos, segmentação de imagens.
- **Processamento de Linguagem Natural (NLP)**: Tradução de texto, classificação de texto, resposta a perguntas.
- **Reconhecimento de Voz**: Identificação de locutores, transcrição de fala.

### Conclusão

Transfer Learning é uma poderosa técnica em Deep Learning que permite reutilizar e adaptar modelos pré-treinados para novas tarefas, economizando tempo e recursos e melhorando o desempenho em tarefas relacionadas.

In [1]:
import os  # Importa o módulo os para interagir com o sistema operacional, como manipulação de diretórios e variáveis de ambiente

# if using Theano with GPU
# os.environ["KERAS_BACKEND"] = "tensorflow"  # Define o backend do Keras para TensorFlow caso esteja usando Theano com GPU (comentado, pois estamos usando TensorFlow por padrão)

import random  # Importa o módulo random para gerar números aleatórios
import numpy as np  # Importa o numpy com o alias np, uma biblioteca fundamental para computação numérica em Python
import keras  # Importa o Keras, uma biblioteca popular de deep learning que fornece uma API simples para construção e treinamento de modelos de redes neurais

import matplotlib.pyplot as plt  # Importa matplotlib.pyplot com o alias plt, uma biblioteca para plotagem de gráficos em Python
from matplotlib.pyplot import imshow  # Importa a função imshow do matplotlib.pyplot para exibir imagens

from keras.preprocessing import image  # Importa submódulos de pré-processamento de imagens do Keras, úteis para carregar e pré-processar imagens
from keras.applications.imagenet_utils import preprocess_input  # Importa preprocess_input de keras.applications.imagenet_utils, uma função para pré-processar imagens de acordo com o padrão do ImageNet
from keras.models import Sequential  # Importa a classe Sequential do Keras para criar modelos sequenciais de redes neurais
from keras.layers import Dense, Dropout, Flatten, Activation  # Importa camadas específicas do Keras usadas na construção de redes neurais
from keras.layers import Conv2D, MaxPooling2D  # Importa camadas convolucionais (Conv2D) e de max pooling (MaxPooling2D), fundamentais para processar dados de imagem em redes convolucionais
from keras.models import Model  # Importa a classe Model do Keras para criar modelos mais complexos e customizados


## Obtendo um Dataset

O primeiro passo é carregar nossos dados. Para este exemplo, utilizaremos o dataset de Gatos e Cachorros, que contém imagens rotuladas de duas categorias: gatos e cachorros. O dataset já está organizado com imagens divididas em pastas para cada classe.

Para obter este dataset, você pode seguir estas etapas:

1. **Baixar o dataset**: Utilize o comando `wget` para baixar o dataset, ou você pode usar o link direto para o download. Aqui estão os comandos:

    ```bash
    wget https://www.microsoft.com/en-us/download/details.aspx?id=54765
    ```

    Se necessário, descompacte o arquivo:

    ```bash
    tar -xvzf nome_do_arquivo.tar.gz
    ```

2. **Organizar o dataset**: Certifique-se de que seu dataset esteja organizado da seguinte forma:
    - O diretório principal deve conter subpastas para cada classe (neste caso, uma subpasta para "gatos" e outra para "cachorros").
    - Cada subpasta deve conter as imagens correspondentes à sua classe.

3. **Carregar o dataset**: Substitua `root` pelo caminho para o seu diretório de dataset. O código abaixo carrega o dataset personalizado, redimensiona as imagens para 224x224 pixels (necessário para a entrada da VGG16), e prepara as imagens para serem usadas no modelo.

    ```python
    root = 'caminho/para/seu/dataset'
    
    # Lista de categorias (pastas) que você deseja incluir no treinamento
    categorias = [x[0] for x in os.walk(root) if x[0]][1:]
    
    # Imprime as categorias disponíveis no dataset
    print(categorias)
    ```

Certifique-se de que a estrutura do diretório esteja correta para que o código funcione corretamente. Se você tiver um estrutura diferente, adapte o código conforme necessário para carregar os dados corretamente.

In [2]:
root = 'kagglecatsanddogs_5340/PetImages'

# Lista de categorias (pastas) que você deseja incluir no treinamento
categories = [os.path.join(root, 'Cat'), os.path.join(root, 'Dog')]

# Imprime as categorias disponíveis no dataset
print(categories)


['kagglecatsanddogs_5340/PetImages/Cat', 'kagglecatsanddogs_5340/PetImages/Dog']


### Função de Pré-processamento de Imagens

Esta função é útil para pré-processar os dados, transformando uma imagem em um vetor de entrada adequado para o modelo.

A função get_image carrega uma imagem, redimensiona-a para 224x224 pixels (o tamanho de entrada esperado pela VGG16), e a prepara para ser alimentada na rede neural. A função retorna a imagem original e o vetor de entrada pré-processado.

In [3]:
# Função auxiliar para carregar a imagem e retornar a imagem e o vetor de entrada
def get_image(path):
    try:
        # Carrega a imagem com o tamanho de entrada esperado (224x224 pixels)
        img = image.load_img(path, target_size=(224, 224))
        
        # Converte a imagem em um array numpy
        x = image.img_to_array(img)
        
        # Adiciona uma dimensão extra ao array para criar um batch do tamanho 1
        x = np.expand_dims(x, axis=0)
        
        # Pré-processa a imagem para o modelo (no caso da VGG16)
        x = preprocess_input(x)
        
        return img, x
    except (IOError, SyntaxError) as e:
        print(f"Erro ao carregar imagem {path}: {e}")
        return None, None

### Carregar Todas as Imagens da Pasta Raiz

Nesta etapa, vamos carregar todas as imagens das pastas de categorias e organizá-las em uma lista. Cada imagem será processada usando a função get_image, e os dados serão armazenados em um formato que inclui a imagem pré-processada e seu rótulo.

In [4]:
# Lista para armazenar os dados das imagens
data = []

# Itera sobre cada categoria
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(category) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg', '.png', '.jpeg']]
    
    # Processa cada imagem
    for img_path in images:
        img, x = get_image(img_path)
        if img is not None and x is not None:
            data.append({'x': np.array(x[0]), 'y': c})

# Conta o número de classes
num_classes = len(categories)

print(f"Total de classes: {num_classes}")

Erro ao carregar imagem kagglecatsanddogs_5340/PetImages/Cat/666.jpg: cannot identify image file <_io.BytesIO object at 0x30d6edf30>


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Erro ao carregar imagem kagglecatsanddogs_5340/PetImages/Dog/11702.jpg: cannot identify image file <_io.BytesIO object at 0x30d602c00>
Total de classes: 2


### Embaralhar os Dados
Essa etapa é importante para garantir que os dados estejam distribuídos aleatoriamente, o que ajuda a evitar vieses durante o treinamento do modelo.

In [7]:
# Embaralha a ordem dos dados
random.shuffle(data)


# Verifica se os dados foram embaralhados corretamente
print(f"Total de dados: {len(data)}")
print(f"Primeiros 5 itens: {data[:5]}")

Total de dados: 24998
Primeiros 5 itens: [{'x': array([[[  53.060997 ,   52.221    ,   49.32     ],
        [  54.060997 ,   53.221    ,   50.32     ],
        [  54.060997 ,   53.221    ,   50.32     ],
        ...,
        [ -76.939    ,  -90.779    ,  -77.68     ],
        [ -80.939    ,  -91.779    ,  -74.68     ],
        [ -80.939    ,  -89.779    ,  -71.68     ]],

       [[  51.060997 ,   50.221    ,   47.32     ],
        [  54.060997 ,   53.221    ,   50.32     ],
        [  57.060997 ,   56.221    ,   53.32     ],
        ...,
        [ -79.939    ,  -90.779    ,  -79.68     ],
        [ -83.939    ,  -92.779    ,  -75.68     ],
        [ -83.939    ,  -91.779    ,  -73.68     ]],

       [[  57.060997 ,   56.221    ,   53.32     ],
        [  57.060997 ,   56.221    ,   53.32     ],
        [  56.060997 ,   55.221    ,   52.32     ],
        ...,
        [ -81.939    ,  -89.779    ,  -81.68     ],
        [ -79.939    ,  -88.779    ,  -71.68     ],
        [ -79.939    ,  -

### Dividir os Dados em Treinamento, Validação e Teste

Nesta etapa, vamos dividir os dados em três conjuntos: treinamento (70%), validação (15%) e teste (15%). Isso nos permitirá **treinar o modelo com o conjunto de treinamento**, **ajustar os hiperparâmetros com o conjunto de validação** e **avaliar o desempenho final com o conjunto de teste.**


In [8]:
# Define as proporções para divisão dos dados
train_split, val_split = 0.7, 0.15

# Calcula os índices para divisão dos dados
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))

# Divide os dados em treinamento, validação e teste
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

# Imprime o tamanho de cada conjunto para verificar a divisão
print(f"Total de dados de treinamento: {len(train)}")
print(f"Total de dados de validação: {len(val)}")
print(f"Total de dados de teste: {len(test)}")

Total de dados de treinamento: 17498
Total de dados de validação: 3750
Total de dados de teste: 3750


### Separar os Dados dos Rótulos

Agora, vamos separar os dados (imagens) dos rótulos para cada um dos conjuntos: treinamento, validação e teste. Isso facilita o uso dos dados nos processos de treinamento e avaliação do modelo.

In [ ]:
# Separa os dados (imagens) dos rótulos para cada conjunto
x_train = np.array([t["x"] for t in train])
y_train = [t["y"] for t in train]

x_val = np.array([t["x"] for t in val])
y_val = [t["y"] for t in val]

x_test = np.array([t["x"] for t in test])
y_test = [t["y"] for t in test]

# Imprime os rótulos do conjunto de teste para verificação
print(y_test)

: 